In [2]:
%%capture
!pip install pretrainedmodels
!pip install torchsummary

In [3]:
import pretrainedmodels
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
import torch

In [13]:
class Resnet34(nn.Module):
    def __init__(self, pretrained=True):
        super(Resnet34, self).__init__()
        model_name = 'resnet34'
        if pretrained is True:
            self.model = pretrainedmodels.__dict__[model_name](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__[model_name](pretrained=None)
            
        self.l0 = nn.Linear(512, 168)
        self.l1 = nn.Linear(512, 11)
        self.l2 = nn.Linear(512, 7)
        
    def forward(self, x):
        bs, c, h, w = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.reshape(bs, -1)
        l0 = self.l0(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        return l0, l1, l2

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(512, 50)
        self.fc2 = nn.Linear(50, 10)
        
        self.l0 = nn.Linear(512, 168)
        self.l1 = nn.Linear(512, 11)
        self.l2 = nn.Linear(512, 7)

    def forward(self, x):
        bs, _, _, _ = x.shape
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.reshape(bs, -1)
        
        l0 = self.l0(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        return l0, l1, l2

In [39]:
class Net(nn.Module):
    """ConvNet -> Max_Pool -> RELU -> ConvNet -> Max_Pool -> RELU -> FC -> RELU -> FC -> SOFTMAX"""
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(5120, 768)
        self.fc2 = nn.Linear(768, 512)
        
        self.l0 = nn.Linear(512, 168)
        self.l1 = nn.Linear(512, 11)
        self.l2 = nn.Linear(512, 7)

    def forward(self, x):
        print(x.shape)
        bs, _, _, _ = x.shape
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.reshape(bs, -1)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        l0 = self.l0(x)
        l1 = self.l1(x)
        l2 = self.l2(x)
        return l0, l1, l2

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = Net().to(device)

summary(model, (1, 137, 236))

torch.Size([2, 1, 137, 236])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 71, 120]             144
            Conv2d-2           [-1, 32, 20, 32]           4,608
            Linear-3                  [-1, 768]       3,932,928
            Linear-4                  [-1, 512]         393,728
            Linear-5                  [-1, 168]          86,184
            Linear-6                   [-1, 11]           5,643
            Linear-7                    [-1, 7]           3,591
Total params: 4,426,826
Trainable params: 4,426,826
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.12
Forward/backward pass size (MB): 1.21
Params size (MB): 16.89
Estimated Total Size (MB): 18.22
----------------------------------------------------------------
